In [1]:
# To install the grein_loader package
#!pip install grein_loader
!pwd

/home/copara/biobombe/0.expression-download


In [27]:
import grein_loader as loader
import pandas as pd
import os
import time

In [23]:
# Load log file from disk to see if this data has already been added
def load_log(logFile):
    log = pd.read_csv(logFile, names=["GSE", "status"])
    log = log[log["status"] == "end"]
    return set(log["GSE"].unique())

In [16]:
def load_data(geo_accession, count_matrix_df):
    description, metadata, count_matrix = loader.load_dataset(geo_accession)
        
    # Merge in to existing count matrix DF
    if count_matrix_df.empty:
        return count_matrix
    else:
        return pd.merge(count_matrix_human_df, count_matrix,  how='left', on=["gene", "gene_symbol"])

In [29]:
# Loading grein overview

overview = loader.load_overview(5)

# Accessing the geo_accession ID and study species from overview
geo_accession_ids = []
species = []
for i in range(len(overview)):
    geo_accession_ids.append(overview[i]['geo_accession'])
    species.append(overview[i]['species'])

In [30]:
# saving the species in each GSE
GSE_species = {'geo_accession_id': geo_accession_ids, 'species': species}
GSE_species_df = pd.DataFrame.from_dict(GSE_species)

In [31]:
# Prepare to download human data

# Filter dataframe for human data only
GSE_species_df_filtered = GSE_species_df[GSE_species_df['species'] == 'Homo sapiens']

GSE_species_df_filtered

# Check if there is already expression data downloaded or whether we are starting fresh
humanLog = "greinLoadHuman.log"
humanData = "download/grein_count_matrix_human.pkl"
if os.path.exists(humanLog) and os.path.exists(humanData): 
    print("...loading human data")
    count_matrix_human_df = pd.read_pickle(humanData)
    completed = load_log(humanLog)
else:
    print("starting human download from scratch")
    count_matrix_human_df = pd.DataFrame()
    completed = set()
    !touch {humanLog}

...loading human data


In [32]:
# Saving human gene expression data
with open(humanLog, "a") as logfile:
    for geo_accession in GSE_species_df_filtered['geo_accession_id']:
        
        # Check if this data was logged as loaded
        if geo_accession in completed:
            print("Already done: {0}".format(geo_accession))
            continue
            
        # If new data, download data and add to df
        print("********* |", geo_accession, "| ***********")
        logfile.write(geo_accession + ",start\n")
        
        count_matrix_human_df = load_data(geo_accession, count_matrix_human_df)
        
        logfile.write(geo_accession + ",write\n")
        count_matrix_human_df.to_pickle(humanData)
        logfile.write(geo_accession + ",end\n")
        


Already done: GSE100007
Already done: GSE100027
Already done: GSE100040
********* | GSE100075 | ***********


KeyboardInterrupt: 

KeyboardInterrupt: 

In [28]:
pd.read_pickle(humanData)

,gene,gene_symbol,GSM2667747,GSM2667748,GSM2667749,GSM2667750,GSM2667751,GSM2667752,GSM2667753,GSM2667754,...,GSM2668086,GSM2668087,GSM2668088,GSM2668089,GSM2671001,GSM2671002,GSM2671003,GSM2671004,GSM2671005,GSM2671006
0,ENSG00000000003,TSPAN6,1766.0677,420.8206,300.8009,3142.8439,2207.8392,4367.5056,228.5759,650.3287,...,254.7605,143.2263,174.3319,202.3253,1053.8788,2405.3001,940.5770,973.7475,760.0100,1472.2712
1,ENSG00000000005,TNMD,43.9280,18.0091,0.0000,115.8917,63.5936,39.7803,11.7588,1.0153,...,0.0000,0.0000,0.0000,0.0000,3.9978,11.0025,4.9979,8.0054,3.9978,11.0011
2,ENSG00000000419,DPM1,1097.7855,367.2333,316.5226,3895.0588,1536.1375,1084.5276,127.7205,245.1014,...,613.1214,374.0965,302.6622,454.0570,1227.1084,2980.0756,1045.8061,1098.7163,759.7729,1398.8077
3,ENSG00000000457,SCYL3,601.4702,270.9239,163.6869,1177.2241,873.9074,845.1423,229.8548,134.4694,...,196.5995,185.3106,260.9485,183.7584,586.7993,1504.0115,619.3764,661.9514,469.5312,1113.1412
4,ENSG00000000460,C1orf112,1040.8861,349.1418,298.1800,1967.3975,2115.2425,980.9359,253.4457,145.7038,...,204.1681,186.2225,127.6457,172.0572,441.0758,1258.3932,466.5542,450.2848,289.2767,467.7875
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27985,ENSG00000283688,MIR6715B,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
27986,ENSG00000283690,MIR3116-2,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
27987,ENSG00000283694,MIR3202-2,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
27988,ENSG00000283697,LOC101928917,1.7315,0.0000,0.0000,0.0000,2.0039,3.9899,2.5184,4.1298,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# Saving mice gene expression data

# Check if there is already data downloaded or whether we are starting fresh
if os.path.exists("greinLoadMice.log") and os.path.exists("download/grein_count_matrix_mice.pkl"): 
    count_matrix_human_df = pd.read_pickle("download/grein_count_matrix_mice.pkl")
    completed = load_log("Mice")

else:
    count_matrix_mice_df = pd.DataFrame()
    completed = set()
    os.system("touch greinLoadMice.log")

with open("greinLoadMice.log", "a") as logfile:
    for geo_accession in geo_accession_ids:
        
        # Check if this data was succesfully pickled
        if geo_accession in completed:
            continue
            
        # If new data, download data and add to df
        print("********* |", geo_accession, "| ***********")
        logfile.write(geo_accession + ",start\n")
        description, metadata, count_matrix = loader.load_dataset(geo_accession)
        # print("Description: ", description)
        
        # Merge in to existing count matrix DF
        if count_matrix_mice_df.empty:
            count_matrix_mice_df = count_matrix
        else:
            count_matrix_mice_df = pd.merge(count_matrix_mice_df, count_matrix,  how='left', on=["gene", "gene_symbol"])
        
        logfile.write(geo_accession + ",write\n")
        count_matrix_mice_df.to_pickle("download/grein_count_matrix_mice.pkl")
        logfile.write(geo_accession + ",end\n")

********* | GSE100007 | ***********
********* | GSE100027 | ***********
********* | GSE100040 | ***********
********* | GSE100075 | ***********
********* | GSE100081 | ***********
********* | GSE100092 | ***********
********* | GSE100099 | ***********
********* | GSE100118 | ***********
********* | GSE100183 | ***********
********* | GSE100206 | ***********
********* | GSE100210 | ***********
********* | GSE100223 | ***********
********* | GSE100258 | ***********
********* | GSE100266 | ***********
********* | GSE100268 | ***********
********* | GSE100291 | ***********
********* | GSE100297 | ***********
********* | GSE100327 | ***********
********* | GSE100338 | ***********
********* | GSE100359 | ***********
********* | GSE100382 | ***********
********* | GSE100392 | ***********
********* | GSE100408 | ***********
********* | GSE100417 | ***********
********* | GSE100427 | ***********
********* | GSE100501 | ***********
********* | GSE100520 | ***********
********* | GSE100530 | ****

In [ ]:
pd.read_pickle("download/grein_count_matrix_human.pkl")


In [ ]:
# Loading data from the data sets using the geo_accession IDs
# NOTE this does not save/pickle the description or metadata at all

with open("greinLoad.log", "a") as logfile:
    for geo_accession in geo_accession_ids:
        
        # Check if this data was succesfully pickled
        if geo_accession in completed:
            continue
            
        # If new data, download data and add to df
        print("********* |", geo_accession, "| ***********")
        logfile.write(geo_accession + ",start\n")
        description, metadata, count_matrix = loader.load_dataset(geo_accession)
        # print("Description: ", description)
        
        # Merge in to existing count matrix DF
        if count_matrix_df.empty:
            count_matrix_df = count_matrix
        else:
            count_matrix_df = pd.merge(count_matrix_df, count_matrix,  how='left', on=["gene", "gene_symbol"])
        
        logfile.write(geo_accession + ",write\n")
        count_matrix_df.to_pickle("download/grein_count_matrix.pkl")
        logfile.write(geo_accession + ",end\n")
        
        break

In [ ]:
# Timimg the loading of data sets
start = time.time()
print(loader.load_overview(10))
end = time.time()
print(end-start)

## Save the data loaded above

In [ ]:
import random
from sklearn.model_selection import train_test_split
import json
import sys

**Count_matrix**

In [ ]:
path = os.path.join(os.getcwd(), 'download', 'count_matrix.tsv')
with open(path, 'w') as file:
    count_matrix_df.to_csv(path, sep="\t") 

**Description**

In [ ]:
# Save description. Currently saves description for only one data set at a time
path = os.path.join('download', 'description.json')
with open(path, 'w') as file:
    # Serialize and write the variable to the file
    json.dump(description, file)

**Metadata**

In [ ]:
# Create a dataframe from metadata
metadata_df = pd.DataFrame.from_dict(metadata)

# list of row names in metadata_df
row_names = list(metadata_df.index)

# Creating a list of rows that we do not require to process this data set
unneeded_rows = ['Consent', ' ', 'channel_count', 'organism_ch1', 'relation', 'status']
for index in row_names:
    if ('contact' in index) or ('date' in index) or ('data_processing' in index) or ('Hash' in index) or ('Date' in index) or ('library' in index):
        unneeded_rows.append(index)
        
# remove unnecessary roles from count_matrix
metadata_df.drop(unneeded_rows, axis = 0)

In [ ]:
print(type(metadata))

In [ ]:
print(type(description))

In [ ]:
print(count_matrix_df)

In [ ]:
print(count_matrix)